In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import cv2
import os
import json
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
mpl.rcParams['figure.figsize'] = (6, 6)
mpl.rcParams['axes.grid'] = False

In [2]:
# df = pd.read_csv('cleaned_imgs_df.csv')
df = pd.read_csv('imputed_imgs_df.csv')

### Test Data Analysis

In [3]:
# Plotting correlation between every pair of columns
# plt.rcParams['figure.figsize'] = [15, 15]
# sns.heatmap(imputed_df.corr(),annot = True,cmap = "YlGnBu")
# plt.title('Correlation Matrix')
# plt.show()

In [4]:
# start_time.describe()

In [5]:
# mpl.rcParams['figure.figsize'] = (6, 6)
# start_time.hist()
# # Start time is from 4 30 am to 7 30 am

In [6]:
# end_time.describe()

In [7]:
# end_time.hist()
# # end time is 8am - 5pm

In [8]:
# df_list[220]

In [9]:
# (df_list[220][df_list[0].columns[2]]).plot()

In [10]:
# length.describe()

## Split data and Normalize

In [11]:
cleaned_df = pd.read_csv('cleaned_df.csv')
imputed_df = pd.read_csv('imputed_df.csv')

In [12]:
from datetime import date
def calc_time(time):
    return int(time.split(':')[0])*60 + int(time.split(':')[1])
def calc_day(day):
    x = date(int(day.split('/')[2]), int(day.split('/')[0]),int(day.split('/')[1]))
    if(day.split('/')[2]=='2017'):
        return x.timetuple().tm_yday-365+1
    if(day.split('/')[2]=='2018'):
        return x.timetuple().tm_yday
    if(day.split('/')[2]=='2019'):
        return x.timetuple().tm_yday+365
    if(day.split('/')[2]=='2020'):
        return x.timetuple().tm_yday+365*2
    if(day.split('/')[2]=='2021'):
        return x.timetuple().tm_yday+365*3+1
def test_calc_day(day):
    x = date(int(day.split('/')[0]), int(day.split('/')[1]),int(day.split('/')[2]))
    if(day.split('/')[0]=='2017'):
        return x.timetuple().tm_yday-365+1
    if(day.split('/')[0]=='2018'):
        return x.timetuple().tm_yday
    if(day.split('/')[0]=='2019'):
        return x.timetuple().tm_yday+365
    if(day.split('/')[0]=='2020'):
        return x.timetuple().tm_yday+365*2
    if(day.split('/')[0]=='2021'):
        return x.timetuple().tm_yday+365*3+1

In [13]:
df_list = [None]*300
for i in range(1,301):
    t_df = pd.read_csv('data_to_nvidia/Data_Test/{}/weather_data.csv'.format(i))
    t_df['Time'] = [calc_time(j) for j in t_df['MST']]
    t_df['day'] = [test_calc_day(j) for j in t_df['DATE (YYYY/MM/DD)']]
    df_list[i-1] = t_df

In [14]:
start_time = []
end_time = []
length = []
test_days_list = []
for t_df in df_list:
    start_time.append(t_df['Time'][0])
    length.append(len(t_df))
    end_time.append(t_df['Time'][len(t_df)-1])
    test_days_list.append(t_df['day'][0])
start_time = pd.Series(start_time)
end_time = pd.Series(end_time)
length = pd.Series(length)

test_days_list_unique = list(pd.Series(test_days_list).unique())

In [15]:
# def fetch(tup):
#     day,start_time,end_time = tup
#     df = imputed_df[imputed_df['day']==day]
#     return df.loc[(df['Time'] >= start_time) & (df['Time'] <= end_time)].reset_index(drop=True)

In [16]:
# Split into Train and Validation data

X = df['day'].unique()
from sklearn.utils import shuffle
X = shuffle(X, random_state = 42)
X = list(X)
for i in test_days_list_unique:
    X.remove(i)
    
train_days = X[0:int(len(X)*0.85)]
val_days = X[int(len(X)*0.85):]
test_days = test_days_list_unique

def get_dataframe(dataf,days_list):
  indexes_list = []
  for day in days_list:
    indexes_list = np.append(indexes_list, dataf[dataf['day'] == day].index)
  return dataf.iloc[np.array(indexes_list).astype('int64')]

In [17]:
# train_df

In [18]:
cols = ['day', 'Normal imgs NE', 'Normal imgs UE','Proj imgs NE', 'Proj imgs UE', 'BRBG imgs', 'CDOC imgs']
train_df = get_dataframe(df, train_days).drop(['MST','Date','MST_img', 'Date_img'], axis = 1)
val_df = get_dataframe(df, val_days).drop(['MST','Date','MST_img', 'Date_img'], axis = 1)
test_df = get_dataframe(df, test_days).drop(['MST','Date','MST_img', 'Date_img'], axis = 1)

a = train_df[cols]
b = val_df[cols]
c = test_df[cols]

In [19]:
# Normalize the data
train_mean = train_df.mean(numeric_only = True)
train_std = train_df.std(numeric_only = True)

train_df = (train_df- train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std
train_df[cols] = a
val_df[cols] = b
test_df[cols] = c

In [20]:
# full_test_dfs = [fetch(i).drop(['MST','Date'], axis = 1) for i in list(zip(test_days_list,start_time,end_time))]

# for i in range(len(full_test_dfs)):
#        full_test_dfs[i] = (full_test_dfs[i]- train_mean) / train_std

In [21]:
def create_submission(x,path):
  x.index = list(range(1,301))
  x.to_csv(path,header=['10_min_horizon','20_min_horizon','30_min_horizon','40_min_horizon','50_min_horizon', '60_min_horizon','70_min_horizon','80_min_horizon','90_min_horizon','100_min_horizon','110_min_horizon','120_min_horizon'], index=True, index_label = 'scenario_set')

In [22]:
#Ensembling

# df_nn = pd.read_csv('/content/sub.csv')
# df_loocf = pd.read_csv('/content/sub3.csv')
# x = (df_nn + df_loocf) / 2
# create_submission(x.drop('scenario_set', axis = 1),'/content/sub1.csv' )

In [23]:
def get_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_LINEAR)
    return img

def get_images_list(paths):
    images_list = []
    for path in paths:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_LINEAR)
        images_list.append(img)
    return np.array(images_list)

In [24]:
# mpl.rcParams['figure.figsize'] = (12, 12)
# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(img)
# axarr[1].imshow(img1)

In [25]:
# mpl.rcParams['figure.figsize'] = (12, 12)
# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(img)
# axarr[1].imshow(img1)

In [26]:
# mpl.rcParams['figure.figsize'] = (12, 12)
# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(img)
# axarr[1].imshow(img1)

In [27]:
# mpl.rcParams['figure.figsize'] = (12, 12)
# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(img)
# axarr[1].imshow(img1)

In [28]:
# mpl.rcParams['figure.figsize'] = (12, 12)
# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(img)
# axarr[1].imshow(img1)

### Using Deep Learning Models

In [29]:
# # extract test data
# import os
# test_data = []*300
# for i in range(1,301):
#   t_df = pd.read_csv(os.path.join('/content/drive/MyDrive/Shell_ai/dataset/test', str(i), "weather_data.csv")) 
#   t_df['Total Cloud Cover [%]'].replace(-7999, np.NaN, inplace  = True)
#   t_df['Total Cloud Cover [%]'].replace(-1, np.NaN, inplace  = True)
#   t_df['Total Cloud Cover [%]'].replace(-6999, np.NaN, inplace  = True) 
#   t_df['Total Cloud Cover [%]'].fillna(method="ffill", inplace=True)
#   t_df['Total Cloud Cover [%]'].fillna(method='backfill', inplace=True)
#   t_df.drop('Time [Mins]', axis = 1, inplace = True)
#   wv = t_df.pop('Peak Wind Speed @ 6ft [m/s]')
#   # Convert to radians.
#   wd_rad = t_df.pop('Avg Wind Direction @ 6ft [deg from N]')*np.pi / 180
#   # Calculate the wind x and y components.
#   t_df['Wind_x'] = wv*np.cos(wd_rad)
#   t_df['Wind_y'] = wv*np.sin(wd_rad)
#   t_df = (t_df- train_mean.drop('day')) / train_std.drop('day')
#   test_data.append(t_df)

In [30]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# tf.random.set_seed(42)
# np.random.seed(42)

In [31]:
non_image_cols1 = ['Albedo',
 'Azimuth Angle',
 'Clear Sky GHI',
 'DNI',
 'Declination Angle',
 'Dew Temp',
 'Dry Temp',
 'GHI',
 'Moisture',
 'Precipitation',
 'Pressure',
 'RH',
 'Snow Depth',
 'TCC',
 'Time',
 'Wet Temp',
 'Wind_x',
 'Wind_y',
 'Zenith Angle',
 'day',
 'distance factor']

non_image_cols = ['Albedo',
 'Azimuth Angle',
 'Clear Sky GHI',
 'DNI',
 'Declination Angle',
 'Dew Temp',
 'Dry Temp',
 'GHI',
 'Moisture',
 'Precipitation',
 'Pressure',
 'RH',
 'Snow Depth',
 'TCC',
 'Wet Temp',
 'Wind_x',
 'Wind_y',
 'day',
 'distance factor']

In [32]:
len(non_image_cols)

19

In [ ]:
a_file = open("all_images.json", "r")
Proj_imgs_NE = a_file.read()
a_file.close()

In [ ]:
json_load =  json.loads(Proj_imgs_NE)

In [33]:
def get_data(ws, dataframe, image_type = ['Proj imgs NE'],dataframe_cols_list = non_image_cols):
    df = dataframe.copy(deep = False)
    df.reset_index(drop = True, inplace = True)
    imgs_df = df[image_type[0]]
    df = df[dataframe_cols_list]
    image_array = np.array([json_load[path] for path in imgs_df ])
    data = []
    images = []
    targets = []
    grouped_df = df.groupby(['day'], sort = False, as_index = False)
    total_window_size = ws + 12
    for day in df['day'].unique():
        day_size = grouped_df.get_group(day)['GHI'].count()
        offset = grouped_df.get_group(day).index.values[0]
        day_df = grouped_df.get_group(day).reset_index(drop = True)
        day_df.drop('day',axis = 1,inplace = True)
        if(day_size<total_window_size):
            pass
        else:
            for i in range(day_size - total_window_size + 1):
                targets.append(np.array(day_df['GHI'].iloc[list(range(ws+i,ws+i+12))]))
                images.append(image_array[offset+i:offset+ws+i])
                data.append(np.array((day_df.iloc[list(range(i,ws+i))])))
    return np.array(data),np.array(images),np.array(targets)

In [ ]:
# X_train,imgs_train,y_train = get_data(12,train_df)

In [ ]:
# X_val,imgs_val,y_val = get_data(12,val_df)

In [ ]:
# X_test,imgs_test,y_test = get_data(12,test_df)

In [ ]:
# np.save('X_train', X_train)
# np.save('imgs_train', imgs_train)
# np.save('y_train', y_train)

# np.save('X_val', X_val)
# np.save('imgs_val', imgs_val)
# np.save('y_val', y_val)

# np.save('X_test', X_test)
# np.save('imgs_test', imgs_test)
# np.save('y_test', y_test)

In [34]:
# loaded_array = np.load('file_name.npy')

X_train = np.load('X_train.npy')
imgs_train = np.load('imgs_train.npy')
y_train = np.load('y_train.npy' )

X_val = np.load('X_val.npy')
imgs_val = np.load('imgs_val.npy')
y_val = np.load('y_val.npy' )

X_test = np.load('X_test.npy')
imgs_test = np.load('imgs_test.npy')
y_test = np.load('y_test.npy' )

In [ ]:
imgs_train.shape

In [ ]:
mean_pixel = []
for c in range(3):
    mean_pixel.append(imgs_train[:,:,:,:,c].mean())

In [ ]:
for c in range(3):
        imgs_train[:,:,:,:,c] = imgs_train[:,:,:,:,c] - mean_pixel[c]

In [ ]:
for c in range(3):
    imgs_val[:,:,:,:,c] = imgs_val[:,:,:,:,c] - mean_pixel[c]

for c in range(3):
    imgs_test[:,:,:,:,c] = imgs_test[:,:,:,:,c] - mean_pixel[c]

In [ ]:
# for c in range(3):
#     print("\n","c =",c )
#     print(imgs_train[:,:,:,:,c].mean())
#     print(imgs_val[:,:,:,:,c].mean())
#     print(imgs_test[:,:,:,:,c].mean())

In [ ]:
# plt.imshow(imgs_train[9,4,:,:,2])

In [ ]:
# day_df = grouped_df.get_group(13).reset_index(drop = True)
# day_df.drop(['day','BRBG imgs','CDOC imgs','Normal imgs NE','Normal imgs UE','Proj imgs UE'],axis = 1,inplace = True)
# day_df

### Wandb

In [ ]:
import wandb
from tensorflow import keras
from wandb.keras import WandbCallback
wandb.login()

In [ ]:
l1 = ['GHI',
 'DNI',
 'Azimuth Angle',
 'Dry Temp',
 'Dew Temp',
 'RH',
 'TCC',
 'Pressure',
 'Precipitation',
 'Snow Depth',
 'Moisture',
 'Albedo',
 'day',
 'Declination Angle',
 'distance factor',
 'Wind_x',
 'Wind_y',
 'Clear Sky GHI']

l2 = ['GHI',
 'DNI',
 'Azimuth Angle',
 'Dry Temp',
 'Dew Temp',
 'RH',
 'TCC',
 'Pressure',
 'Precipitation',
 'Moisture',
 'Wind_x',
 'Wind_y',
 'distance factor',
 'Clear Sky GHI']

l3 = ['GHI',
       'DNI',
 'Azimuth Angle',
 'Dry Temp',
 'RH',
 'Pressure',
 'TCC',
 'Precipitation',
 'distance factor',
 'Clear Sky GHI']


l4 = ['GHI',
 'Azimuth Angle',
 'TCC',
 'distance factor',
 'Clear Sky GHI']

l = [l1,l2,l3,l4]


### MLP

In [46]:
len(non_image_cols)

19

In [ ]:
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [35]:
mlp_input = tf.keras.layers.Input(shape=(12,len(non_image_cols)-1 ))
# input2 = tf.keras.layers.Input(shape=(12,64,64,3))
### CNN ###
# x = tf.keras.layers.Conv2D(64, (3, 3), padding = "same", activation='relu', input_shape=(64, 64, 3))(input2)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)
# x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3))(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)
# x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)
# x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)
# x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(units=256,activation='relu')(x)
# x = tf.keras.layers.Dense(units=256,activation='relu')(x)

### MLP ###
t = tf.keras.layers.Flatten()(mlp_input)
t = tf.keras.layers.Dense(units=256,activation='leaky_relu')(t)
t= tf.keras.layers.Dense(units=256,activation='leaky_relu')(t)
t = tf.keras.layers.Dense(units=32,activation='leaky_relu')(t)
# t = tf.keras.layers.Dense(units=12,activation='leaky_relu')(t)

# combinedInput = concatenate([x, t])
# f = tf.keras.layers.Dense(256, activation="relu")(combinedInput)
# f = tf.keras.layers.Dense(64, activation="relu")(combinedInput)
# f = tf.keras.layers.Dense(12, activation="linear")(x)

mlpmodel = tf.keras.Model(inputs=mlp_input, outputs=t)

2021-12-03 23:35:06.604197: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2021-12-03 23:35:06.604910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14659 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:07:00.0, compute capability: 7.0
2021-12-03 23:35:06.606534: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 1
2021-12-03 23:35:06.607116: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14659 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB, pci bus id: 0000:0a:00.0, compute capability: 7.0


In [47]:
!nvidia-smi

Sat Dec  4 00:35:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   36C    P0    58W / 300W |   9626MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   

In [37]:
conv_input = tf.keras.layers.Input(shape=(64,64,3))
### CNN ###
x = tf.keras.layers.Conv2D(64, (3, 3), padding = "same", activation='relu', input_shape=(64, 64, 3))(conv_input)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3))(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units=256,activation='relu')(x)
x = tf.keras.layers.Dense(units=256,activation='relu')(x)

cnnmodel = tf.keras.Model(inputs=conv_input, outputs=x)

In [38]:
input1 = tf.keras.layers.Input(shape=(12,len(non_image_cols)-1 ))
input2 = tf.keras.layers.Input(shape=(12,64,64,3))

x = tf.keras.layers.TimeDistributed(cnnmodel, input_shape=(12,64,64,3))(input2)
x = tf.keras.layers.Flatten()(x)
y = mlpmodel(input1)

combinedInput = tf.keras.layers.Concatenate()([x, y])
f = tf.keras.layers.Dense(256, activation="relu")(combinedInput)
# f = tf.keras.layers.Dropout(.5)(f)
f = tf.keras.layers.Dense(64, activation="relu")(combinedInput)
f = tf.keras.layers.Dense(12, activation="linear")(x)
mixed_model = tf.keras.Model(inputs=[input1,input2], outputs=f)

12.459308624267578

In [48]:
callbacks = [
      EarlyStopping(monitor='val_loss', patience=9,  mode='min',verbose=1),
      ReduceLROnPlateau(factor=0.15, patience=5, min_lr=1e-8, verbose=1),
      ModelCheckpoint('Mixed_model.h5', verbose=1, save_best_only=True, save_weights_only=True) ]

mixed_model.compile(loss=tf.keras.losses.Huber(),
                optimizer=tf.optimizers.Adam(learning_rate=1e-3),
                metrics=[tf.metrics.MeanAbsoluteError()])

history = mixed_model.fit(x=[X_train,imgs_train][:10000], y=y_train[:10000],
                          validation_data=([X_val, imgs_val],y_val),
     epochs=1, batch_size=1,  shuffle=True)

# model.evaluate(test_gen)

2021-12-04 00:36:03.321851: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 47597617152 exceeds 10% of free system memory.
2021-12-04 00:36:32.734019: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:265] gpu_async_0 cuMemAllocAsync failed to allocate 47597617152 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 6851133440/16945512448
2021-12-04 00:36:32.734794: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:270] Stats: Limit:                     15371862016
InUse:                      8965875340
MaxInUse:                   9051974952
NumAllocs:                     1509575
MaxAllocSize:               8668053504
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2021-12-04 00:36:32.734830: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:56] Histogram of current allocation: (allocati

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
callbacks = [
      EarlyStopping(monitor='val_loss', patience=9,  mode='min',verbose=1),
      ReduceLROnPlateau(factor=0.15, patience=5, min_lr=1e-8, verbose=1),
      ModelCheckpoint('Mixed_model.h5', verbose=1, save_best_only=True, save_weights_only=True) ]

model.compile(loss=tf.keras.losses.Huber(),
                optimizer=tf.optimizers.Adam(learning_rate=1e-3),
                metrics=[tf.metrics.MeanAbsoluteError()])

history = model.fit(x=[X_train, imgs_train], y=y_train,
     validation_data=([X_val, imgs_val], y_val),
     epochs=20, batch_size=128 callbacks=callbacks)

# model.evaluate(test_gen)

In [ ]:
def get_test_predictions(model, ws,features_list,sr) :

  X = np.zeros((300,ws,len(features_list)))
  for i in range(300):  
    X[i] = np.array(full_test_dfs[i][(len(full_test_dfs[i])-1)%sr::sr][-ws:][features_list])
  output = model.predict(X)
  output = output.reshape(300,12)
  output = pd.DataFrame(output)
#     print(output)
  output = output*train_std.loc['GHI'] + train_mean.loc['GHI']
  for i in range(300):
    for j in range(4):
      if(output.iloc[i][j] < -1):
          output.iloc[i][j] = -0.8
  create_submission(pd.DataFrame(np.array(output)),'predictions_ws_{}sr_{}cols_{}.csv'.format(ws,sr,len(features_list)))
  return output

In [ ]:
model.built = True
model.load_weights('MLP_l:0.01_sr:10_ws:12_u1:256_u2:32_cols:2.h5')
model.evaluate(test_gen, steps = test_batches)

In [ ]:
get_test_predictions(model,12,l[2],10)

In [ ]:
mpl.rcParams['figure.figsize'] = (8,8)
day = 100
grouped_imputed_df = imputed_df.groupby(['day'], sort = False, as_index = False)
def plot(i):
    pd.Series(x.iloc[i].values , index=range(len(full_test_dfs[i])+10,len(full_test_dfs[i])+121,10)).plot()
    (grouped_imputed_df.get_group(test_days_list[i])['GHI'].reset_index(drop = True)).plot()

### CNN

In [ ]:
def Model(filters,kernel_size,ws,i,units_1):

    model =   tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=filters,kernel_size=kernel_size,activation='relu'
                               ,input_shape=(ws,len(l[i]))),
        tf.keras.layers.MaxPooling1D(pool_size = 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=units_1, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(units=12),
    ])
    model.summary()
    return model

# model = Sequential()
# model.add(Conv1D(filters=32, kernel_size=3, activation= ' relu ' ,
# input_shape=(n_timesteps,n_features)))
# model.add(Conv1D(filters=32, kernel_size=3, activation= ' relu ' ))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=16, kernel_size=3, activation= ' relu ' ))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(100, activation= ' relu ' ))
model.add(Dense(n_outputs))